<a href="https://colab.research.google.com/github/nadiazk/Machine-Learning-with-Python/blob/master/Data_jsontoDWH_Schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transaction Payload Json reader for DWH Schema Development

This notebook works on Monzo and Lloyds transaction jsons, adds Application and user attributes and combines them into one master json for one user with multiple accounts

Authenticating and setting Project ID

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[categorization-250610]' #This is our project for categorization

Retrieving jsons of one user and different accounts from GCP Bucket

In [4]:
!gsutil cp gs://raw_jsons_data_collection/response_008_NZ_132_monzo.json ./
!gsutil cp gs://raw_jsons_data_collection/response_008_NZ_197_lloyds.json ./

Copying gs://raw_jsons_data_collection/response_008_NZ_132_monzo.json...
/ [1 files][ 41.9 KiB/ 41.9 KiB]                                                
Operation completed over 1 objects/41.9 KiB.                                     
Copying gs://raw_jsons_data_collection/response_008_NZ_197_lloyds.json...
/ [1 files][127.4 KiB/127.4 KiB]                                                
Operation completed over 1 objects/127.4 KiB.                                    


In [0]:
#Importing relevant Libraries
import pandas as pd
import json
from pandas.io.json import json_normalize

## Monzo

Reading File and Normalizing JSON into a Dataframe

In [36]:
file = 'response_008_NZ_132_monzo.json'#enter json file name here

with open(file) as json_file:
    data = json.load(json_file)
    df_1 = json_normalize(data['data'])
df_1.info()# 11 columns imported from monzo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 11 columns):
id                            132 non-null object
date                          132 non-null object
bookingDateTime               132 non-null object
status                        132 non-null object
amount                        132 non-null float64
currency                      132 non-null object
reference                     132 non-null object
description                   132 non-null object
transactionAmount.amount      132 non-null float64
transactionAmount.currency    132 non-null object
valueDateTime                 94 non-null object
dtypes: float64(2), object(9)
memory usage: 11.5+ KB


In [37]:
df_1.head()

,id,date,bookingDateTime,status,amount,currency,reference,description,transactionAmount.amount,transactionAmount.currency,valueDateTime
0,tx_00009oQ1GJHfM97IUC9C8g,2019-10-29T13:14:11.9Z,2019-10-29T13:14:11.9Z,PENDING,-17.00,GBP,,THE SHOREDITCH SHOP LONDON GBR,-17.00,GBP,NaN
1,tx_00009oPeyzF0QMppTbu5Rp,2019-10-29T09:04:33.592Z,2019-10-29T09:04:33.592Z,PENDING,-12.43,GBP,,PRET A MANGER LONDON GBR,-12.43,GBP,NaN
2,tx_00009oNb8xTh2uUWOZCZRR,2019-10-28T09:12:07.086Z,2019-10-28T09:12:07.086Z,BOOKED,-15.49,GBP,,PRET A MANGER LONDON GBR,-15.49,GBP,2019-10-29T11:22:16.071Z
3,tx_00009oK5mtO7tJ7jdZzioE,2019-10-26T16:36:38.576Z,2019-10-26T16:36:38.576Z,BOOKED,-0.89,GBP,,TESCO STORES 4447 BAYSWATER GBR,-0.89,GBP,2019-10-28T05:10:51.779Z
4,tx_00009oK5MrtAqIwrSU6j7h,2019-10-26T16:31:56.031Z,2019-10-26T16:31:56.031Z,BOOKED,-6.45,GBP,,PATISSERIE VALERIE LONDON GBR,-6.45,GBP,2019-10-28T03:34:26.486Z


Monzo has transactions IDs as 'id' coming in

Adding User identifiers from "get users" from GoSafeConnect R&D app for Monzo jsons



In [38]:
df_1['userUuid'] = "50b7d064-f078-4cdb-baa6-5abe91080cd5"
df_1['applicationUuid'] = "8a2cd54a-5976-4d11-8a0f-92079b35dada"
df_1['applicationName'] = "GoSafeConnectRandD"
df_1['applicationUserId'] = "008"
df_1["institutionConsents.institutionId"] = "monzo"
df_1.head()

,id,date,bookingDateTime,status,amount,currency,reference,description,transactionAmount.amount,transactionAmount.currency,valueDateTime,userUuid,applicationUuid,applicationName,applicationUserId,institutionConsents.institutionId
0,tx_00009oQ1GJHfM97IUC9C8g,2019-10-29T13:14:11.9Z,2019-10-29T13:14:11.9Z,PENDING,-17.00,GBP,,THE SHOREDITCH SHOP LONDON GBR,-17.00,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo
1,tx_00009oPeyzF0QMppTbu5Rp,2019-10-29T09:04:33.592Z,2019-10-29T09:04:33.592Z,PENDING,-12.43,GBP,,PRET A MANGER LONDON GBR,-12.43,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo
2,tx_00009oNb8xTh2uUWOZCZRR,2019-10-28T09:12:07.086Z,2019-10-28T09:12:07.086Z,BOOKED,-15.49,GBP,,PRET A MANGER LONDON GBR,-15.49,GBP,2019-10-29T11:22:16.071Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo
3,tx_00009oK5mtO7tJ7jdZzioE,2019-10-26T16:36:38.576Z,2019-10-26T16:36:38.576Z,BOOKED,-0.89,GBP,,TESCO STORES 4447 BAYSWATER GBR,-0.89,GBP,2019-10-28T05:10:51.779Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo
4,tx_00009oK5MrtAqIwrSU6j7h,2019-10-26T16:31:56.031Z,2019-10-26T16:31:56.031Z,BOOKED,-6.45,GBP,,PATISSERIE VALERIE LONDON GBR,-6.45,GBP,2019-10-28T03:34:26.486Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo


Adding account related identifiers and parameters from the "get Accounts" API response



In [39]:
df_1["accountId"] = "acc_00009jPOqnUeO6s7BWk0w5"
df_1["account.currency"] = "GBP"
df_1["account.usageType"]= "PERSONAL"
df_1["accountType"] = "CURRENT"
df_1['accounts.meta.count'] = "1"#How and where to add this?
#df_1["transactionHistory_months"] = df_1['date'][0]- df_1['date'][131]
df_1['institutionConsents.count'] = '2'
df_1.head()

,id,date,bookingDateTime,status,amount,currency,reference,description,transactionAmount.amount,transactionAmount.currency,valueDateTime,userUuid,applicationUuid,applicationName,applicationUserId,institutionConsents.institutionId,accountId,account.currency,account.usageType,accountType,accounts.meta.count,institutionConsents.count
0,tx_00009oQ1GJHfM97IUC9C8g,2019-10-29T13:14:11.9Z,2019-10-29T13:14:11.9Z,PENDING,-17.00,GBP,,THE SHOREDITCH SHOP LONDON GBR,-17.00,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
1,tx_00009oPeyzF0QMppTbu5Rp,2019-10-29T09:04:33.592Z,2019-10-29T09:04:33.592Z,PENDING,-12.43,GBP,,PRET A MANGER LONDON GBR,-12.43,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
2,tx_00009oNb8xTh2uUWOZCZRR,2019-10-28T09:12:07.086Z,2019-10-28T09:12:07.086Z,BOOKED,-15.49,GBP,,PRET A MANGER LONDON GBR,-15.49,GBP,2019-10-29T11:22:16.071Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
3,tx_00009oK5mtO7tJ7jdZzioE,2019-10-26T16:36:38.576Z,2019-10-26T16:36:38.576Z,BOOKED,-0.89,GBP,,TESCO STORES 4447 BAYSWATER GBR,-0.89,GBP,2019-10-28T05:10:51.779Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
4,tx_00009oK5MrtAqIwrSU6j7h,2019-10-26T16:31:56.031Z,2019-10-26T16:31:56.031Z,BOOKED,-6.45,GBP,,PATISSERIE VALERIE LONDON GBR,-6.45,GBP,2019-10-28T03:34:26.486Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2


Standardizing the 'id' field that has Transaction IDs coming from Monzo to the 'Transaction IDs'

In [40]:
df_1.columns=[it.replace('id','transactionId') for it in df_1.columns]
df_1.tail()

,transactionId,date,bookingDateTime,status,amount,currency,reference,description,transactionAmount.amount,transactionAmount.currency,valueDateTime,userUutransactionId,applicationUutransactionId,applicationName,applicationUserId,institutionConsents.institutionId,accountId,account.currency,account.usageType,accountType,accounts.meta.count,institutionConsents.count
127,tx_00009jZMYhAktSDo6e1y9i,2019-06-06T12:04:42.901Z,2019-06-06T12:04:42.901Z,PENDING,0.0,GBP,Active card check,UberBV help.uber.com NLD,0.0,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
128,tx_00009jYBl1B335Bgvw9uG9,2019-06-05T22:28:59.019Z,2019-06-05T22:28:59.019Z,PENDING,0.0,GBP,Active card check,GOOGLE *SERVICES g.co/helppay# GBR,0.0,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
129,tx_00009jXOQrIJBiQajjJQfJ,2019-06-05T13:16:18.032Z,2019-06-05T13:16:18.032Z,BOOKED,99.0,GBP,,Cash deposit,99.0,GBP,2019-06-11T00:00:00Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
130,tx_00009jXOMqRwO0vKvls39d,2019-06-05T13:15:34.524Z,2019-06-05T13:15:34.524Z,BOOKED,299.0,GBP,,Cash deposit,299.0,GBP,2019-06-11T00:00:00Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2
131,tx_00009jXNlqqlvzjRSTqUN7,2019-06-05T13:08:53.382Z,2019-06-05T13:08:53.382Z,BOOKED,299.0,GBP,,Cash deposit,299.0,GBP,2019-06-11T00:00:00Z,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,GoSafeConnectRandD,008,monzo,acc_00009jPOqnUeO6s7BWk0w5,GBP,PERSONAL,CURRENT,1,2


# Lloyds

Reading File and Normalizing JSON into a Dataframe

In [41]:
file = 'response_008_NZ_197_lloyds.json'#enter json file name here

with open(file) as json_file:
    data = json.load(json_file)
    df_1_2 = json_normalize(data['data'])
df_1_2.info()# 22 columns imported from lloyds

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 22 columns):
date                                         197 non-null object
bookingDateTime                              197 non-null object
status                                       197 non-null object
amount                                       197 non-null float64
currency                                     197 non-null object
transactionAmount.amount                     197 non-null float64
transactionAmount.currency                   197 non-null object
proprietaryBankTransactionCode.code          197 non-null object
proprietaryBankTransactionCode.issuer        197 non-null object
valueDateTime                                196 non-null object
description                                  196 non-null object
transactionInformation                       196 non-null object
isoBankTransactionCode.domainCode.code       196 non-null object
isoBankTransactionCode.domainCode.name       196

Adding User identifiers from "get users" from GoSafeConnect R&D app for Lloyds jsons

Lloyds has 3 level iso codes and their names coming in as well

In [0]:
df_1_2['userUuid'] = "50b7d064-f078-4cdb-baa6-5abe91080cd5"
df_1_2['applicationUuid'] = "8a2cd54a-5976-4d11-8a0f-92079b35dada"
df_1_2['applicationUserId'] = "008"
df_1_2['applicationName'] = "GoSafeConnectRandD"
df_1_2['transactionId'] = "TBA"
df_1_2["institutionConsents.institutionId"] = "lloyds"


In [43]:
df_1_2.head()

,date,bookingDateTime,status,amount,currency,transactionAmount.amount,transactionAmount.currency,proprietaryBankTransactionCode.code,proprietaryBankTransactionCode.issuer,valueDateTime,description,transactionInformation,isoBankTransactionCode.domainCode.code,isoBankTransactionCode.domainCode.name,isoBankTransactionCode.familyCode.code,isoBankTransactionCode.familyCode.name,isoBankTransactionCode.subFamilyCode.code,isoBankTransactionCode.subFamilyCode.name,balance.type,balance.balanceAmount.amount,balance.balanceAmount.currency,reference,userUuid,applicationUuid,applicationUserId,applicationName,transactionId,institutionConsents.institutionId
0,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,PENDING,-11.29,GBP,-11.29,GBP,DEB,LBG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds
1,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-44.75,GBP,-44.75,GBP,DEB,LBG,2019-11-07T00:00:00Z,SAINSBURYS S/MKTS,[SAINSBURYS S/MKTS],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,591.72,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds
2,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-12.20,GBP,-12.20,GBP,DEB,LBG,2019-11-07T00:00:00Z,COSTA COFFEE,[COSTA COFFEE],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,636.47,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds
3,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-7.99,GBP,-7.99,GBP,DEB,LBG,2019-11-07T00:00:00Z,AMAZON PRIME*IW4VV,[AMAZON PRIME*IW4VV],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,648.67,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds
4,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-20.00,GBP,-20.00,GBP,DEB,LBG,2019-11-07T00:00:00Z,LUL TICKET MACHINE,[LUL TICKET MACHINE],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,656.66,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds


Adding account related identifiers and parameters from the "get Accounts" API response`

In [0]:
df_1_2["accountId"] = "25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8"
df_1_2["account.currency"] = "GBP"
df_1_2["account.usageType"]= "PERSONAL"
df_1_2["accountType"] = "CURRENT"
df_1_2['accounts.meta.count'] = "1"#How and where to add this?
#df_1["transactionHistory_months"] = df_1['date'][0]- df_1['date'][131]
df_1_2['institutionConsents.count'] = '2'

In [45]:
df_1_2.head()

,date,bookingDateTime,status,amount,currency,transactionAmount.amount,transactionAmount.currency,proprietaryBankTransactionCode.code,proprietaryBankTransactionCode.issuer,valueDateTime,description,transactionInformation,isoBankTransactionCode.domainCode.code,isoBankTransactionCode.domainCode.name,isoBankTransactionCode.familyCode.code,isoBankTransactionCode.familyCode.name,isoBankTransactionCode.subFamilyCode.code,isoBankTransactionCode.subFamilyCode.name,balance.type,balance.balanceAmount.amount,balance.balanceAmount.currency,reference,userUuid,applicationUuid,applicationUserId,applicationName,transactionId,institutionConsents.institutionId,accountId,account.currency,account.usageType,accountType,accounts.meta.count,institutionConsents.count
0,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,PENDING,-11.29,GBP,-11.29,GBP,DEB,LBG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds,25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8,GBP,PERSONAL,CURRENT,1,2
1,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-44.75,GBP,-44.75,GBP,DEB,LBG,2019-11-07T00:00:00Z,SAINSBURYS S/MKTS,[SAINSBURYS S/MKTS],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,591.72,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds,25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8,GBP,PERSONAL,CURRENT,1,2
2,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-12.20,GBP,-12.20,GBP,DEB,LBG,2019-11-07T00:00:00Z,COSTA COFFEE,[COSTA COFFEE],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,636.47,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds,25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8,GBP,PERSONAL,CURRENT,1,2
3,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-7.99,GBP,-7.99,GBP,DEB,LBG,2019-11-07T00:00:00Z,AMAZON PRIME*IW4VV,[AMAZON PRIME*IW4VV],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,648.67,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds,25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8,GBP,PERSONAL,CURRENT,1,2
4,2019-11-07T00:00:00Z,2019-11-07T00:00:00Z,BOOKED,-20.00,GBP,-20.00,GBP,DEB,LBG,2019-11-07T00:00:00Z,LUL TICKET MACHINE,[LUL TICKET MACHINE],PMNT,Payments,CCRD,Customer Card Transactions,POSD,Point-of-Sale (POS) Payment - Debit Card,INTERIM_BOOKED,656.66,GBP,NaN,50b7d064-f078-4cdb-baa6-5abe91080cd5,8a2cd54a-5976-4d11-8a0f-92079b35dada,008,GoSafeConnectRandD,TBA,lloyds,25cbdf6c-00d2-3515-bdd1-ae9134a3f5f8,GBP,PERSONAL,CURRENT,1,2


# Joining one user's multiple accounts into a single data frame

In [0]:
oneuser_multiAccount_DF = pd.concat([df_1, df_1_2], sort=False, axis =0)

In [57]:
oneuser_multiAccount_DF.info()
#Master Schema Looks Like this now with 36 Columns in total

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 196
Data columns (total 36 columns):
transactionId                                329 non-null object
date                                         329 non-null object
bookingDateTime                              329 non-null object
status                                       329 non-null object
amount                                       329 non-null float64
currency                                     329 non-null object
reference                                    138 non-null object
description                                  328 non-null object
transactionAmount.amount                     329 non-null float64
transactionAmount.currency                   329 non-null object
valueDateTime                                290 non-null object
userUutransactionId                          132 non-null object
applicationUutransactionId                   132 non-null object
applicationName                              329

Validation: Understanding the differences between the datasets

Final Schema vs. Monzo payload difference:

In [48]:
oneuser_multiAccount_DF.columns.difference(df_1.columns)


Index(['applicationUuid', 'balance.balanceAmount.amount',
       'balance.balanceAmount.currency', 'balance.type',
       'isoBankTransactionCode.domainCode.code',
       'isoBankTransactionCode.domainCode.name',
       'isoBankTransactionCode.familyCode.code',
       'isoBankTransactionCode.familyCode.name',
       'isoBankTransactionCode.subFamilyCode.code',
       'isoBankTransactionCode.subFamilyCode.name',
       'proprietaryBankTransactionCode.code',
       'proprietaryBankTransactionCode.issuer', 'transactionInformation',
       'userUuid'],
      dtype='object')

In [49]:
oneuser_multiAccount_DF.columns.difference(df_1_2.columns)

Index(['applicationUutransactionId', 'userUutransactionId'], dtype='object')

# Saving the compiled in to a CSV

In [0]:
oneuser_multiAccount_DF.to_csv ('008_lloyds_Monzo_master.csv')

#Pushing CSV and HTML reports to GCP Bucket


This command will push it to GCP in the same bucket's data_output folder

In [0]:
%%capture
!gsutil cp 008_lloyds_Monzo_master.csv gs://raw_jsons_data_collection/data_output/008_lloyds_Monzo_master.csv

In [58]:
import pandas_profiling
pandas_profiling.ProfileReport(oneuser_multiAccount_DF)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,37
Number of observations,329
Total Missing (%),20.4%
Total size in memory,95.2 KiB
Average record size in memory,296.4 B
Numeric,3
Categorical,23
Boolean,0
Date,0
Text (Unique),0
Rejected,10


In [60]:
profile = pandas_profiling.ProfileReport(oneuser_multiAccount_DF)
profile.to_file(outputfile="./report_008_lloyds_Monzo_master.html")

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [0]:
%%capture
!gsutil cp report_008_lloyds_Monzo_master.html gs://raw_jsons_data_collection/data_output/reports/report_008_lloyds_Monzo_master.html